# Рекомендация тарифов

Продолжение предыдущего проекта. В этот раз необходимо построить модель для задачи классификации, которая позволит предложить клиентам подходящий тариф на основе их поведения. Требуется довести долю правильных ответов (accuracy) до 0.75.

Для поиска оптимального тарифа будем использовать:
- DecisionTreeClassifier
- RandomForestClassifier
- LogisticRegression

### План решения задачи

- [Шаг 1. Загрузка и изучение данных](#1-bullet)
- [Шаг 2. Разбиение исходных данных](#2-bullet)
- [Шаг 3. Тест моделей](#3-bullet)
- [Шаг 4. Проверка модели на тестовой выборке](#4-bullet)
- [Шаг 5. Проверка модели на адекватность](#5-bullet)

## Шаг 1. Загрузка и изучение данных <a id='1-bullet'></a>

In [94]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score

In [95]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [96]:
df.sample(5)

,calls,minutes,messages,mb_used,is_ultra
6,57.0,431.64,20.0,3738.90,1
2752,63.0,478.92,54.0,23681.93,1
896,86.0,632.16,47.0,18910.08,0
2358,77.0,695.53,0.0,19981.20,1
2082,56.0,368.11,36.0,16003.15,0


In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [98]:
df.is_ultra.value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Целевой признак (который будем предсказывать) - **is_ultra**:
- 0 - если клиенту нужно предложить тариф "Смарт"
- 1 - если нужно предложить тариф "Ультра"

Классы не сбалансированы - попробуем это учесть

Предсказвать будем по следующим признакам:
- **сalls** — количество звонков,
- **minutes** — суммарная длительность звонков в минутах,
- **messages** — количество sms-сообщений,
- **mb_used** — израсходованный интернет-трафик в Мб

## Шаг 2. Разбиение исходных данных <a id='2-bullet'></a>

In [99]:
# 60% на обучающую выборку

df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345)

# 20% на валидационную и 20% на тестовую

df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=12345)

In [100]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

In [101]:
print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)

(1928, 4)
(1928,)
(643, 4)
(643,)


## 3. Тест моделей <a id='3-bullet'></a>

### Дерево классификации

In [102]:
max_depths = range(5, 10, 2)
min_samples_leafs = [50, 100, 150]

for max_depth in max_depths:
    for min_samples_leaf in min_samples_leafs:
        tree = DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=min_samples_leaf, random_state=12345)
        tree.fit(features_train, target_train)
        tree_predictions_train = tree.predict(features_train)
        tree_predictions_valid = tree.predict(features_valid)
        print("Training Accuracy: %.5f" %accuracy_score(target_train, tree_predictions_train))
        print("Valid Accuracy: %.5f" %accuracy_score(target_valid, tree_predictions_valid))
        print('max_depth: ', max_depth)
        print('sample_leafs: ', min_samples_leaf)
        print('\n')        

Training Accuracy: 0.80757
Valid Accuracy: 0.78538
max_depth:  5
sample_leafs:  50


Training Accuracy: 0.79772
Valid Accuracy: 0.76983
max_depth:  5
sample_leafs:  100


Training Accuracy: 0.78320
Valid Accuracy: 0.78227
max_depth:  5
sample_leafs:  150


Training Accuracy: 0.81691
Valid Accuracy: 0.78694
max_depth:  7
sample_leafs:  50


Training Accuracy: 0.79772
Valid Accuracy: 0.76983
max_depth:  7
sample_leafs:  100


Training Accuracy: 0.78320
Valid Accuracy: 0.78227
max_depth:  7
sample_leafs:  150


Training Accuracy: 0.81691
Valid Accuracy: 0.78694
max_depth:  9
sample_leafs:  50


Training Accuracy: 0.79772
Valid Accuracy: 0.76983
max_depth:  9
sample_leafs:  100


Training Accuracy: 0.78320
Valid Accuracy: 0.78227
max_depth:  9
sample_leafs:  150




Наилучшее качество модели на валидационной выборке: 0.78694 при значениях max_depth = 7 или 9, sample_leafs = 50

### Случайный лес

In [103]:
# честно говоря так и не понял почему у меня никак не получалось итерироваться по n_estims в таком формате
# то ругался на range, то на list

max_depths = range(5, 10, 2)
n_estims = range(10, 101, 10)

for max_depth in max_depths:
    for n_estimator in n_estims:
        forest = RandomForestClassifier(n_estimators=n_estims, max_depth=max_depth, random_state=12345)
        forest.fit(features_train, target_train)
        predictions_train = forest.predict(features_train)
        predictions_valid = forest.predict(features_valid)
        print("Training Accuracy: %.5f" %accuracy_score(target_train, predictions_train))
        print("Valid Accuracy: %.5f" %accuracy_score(target_valid, predictions_valid))
        print('max_depth: ', max_depth)
        print('n_estimator: ', n_estims)
        print('\n')        

ValueError: n_estimators must be an integer, got <class 'range'>.

In [104]:
for n_estims in range(10, 101, 10):
    for max_depth in range(5, 10, 2):
        forest = RandomForestClassifier(max_depth=max_depth, n_estimators=n_estims, random_state=12345)
        forest.fit(features_train, target_train)
        forest_predictions_train = forest.predict(features_train)
        forest_predictions_valid = forest.predict(features_valid)
        print("Training Accuracy: %.5f" %accuracy_score(target_train, forest_predictions_train))
        print("Valid Accuracy: %.5f" %accuracy_score(target_valid, forest_predictions_valid))
        print('max_depth: ', max_depth)
        print('n_estimator: ', n_estims)
        print('\n')        

Training Accuracy: 0.82417
Valid Accuracy: 0.79316
max_depth:  5
n_estimator:  10


Training Accuracy: 0.85529
Valid Accuracy: 0.79471
max_depth:  7
n_estimator:  10


Training Accuracy: 0.87293
Valid Accuracy: 0.78538
max_depth:  9
n_estimator:  10


Training Accuracy: 0.82624
Valid Accuracy: 0.79005
max_depth:  5
n_estimator:  20


Training Accuracy: 0.85892
Valid Accuracy: 0.80093
max_depth:  7
n_estimator:  20


Training Accuracy: 0.88330
Valid Accuracy: 0.79005
max_depth:  9
n_estimator:  20


Training Accuracy: 0.82417
Valid Accuracy: 0.79316
max_depth:  5
n_estimator:  30


Training Accuracy: 0.85788
Valid Accuracy: 0.80249
max_depth:  7
n_estimator:  30


Training Accuracy: 0.88382
Valid Accuracy: 0.79316
max_depth:  9
n_estimator:  30


Training Accuracy: 0.82624
Valid Accuracy: 0.79471
max_depth:  5
n_estimator:  40


Training Accuracy: 0.86203
Valid Accuracy: 0.80249
max_depth:  7
n_estimator:  40


Training Accuracy: 0.88485
Valid Accuracy: 0.79471
max_depth:  9
n_estimator

Наилучшее качество модели на валидационной выборке: 0.80249 при значениях max_depth = 7, n_estimator =  50

### Логистическая регрессия

In [105]:
for solver in ['newton-cg', 'lbfgs', 'liblinear']:
    lg_reg = LogisticRegression(solver=solver, max_iter=4000, random_state=12345)
    lg_reg.fit(features_train, target_train)
    lg_reg_predictions_train = lg_reg.predict(features_train)
    lg_reg_predictions_valid = lg_reg.predict(features_valid)
    print("Training Accuracy: %.5f" %accuracy_score(target_train, lg_reg_predictions_train))
    print("Valid Accuracy: %.5f" %accuracy_score(target_valid, lg_reg_predictions_valid))
    print('solver: ', solver)
    print('\n')        

Training Accuracy: 0.75311
Valid Accuracy: 0.75583
solver:  newton-cg


Training Accuracy: 0.71317
Valid Accuracy: 0.71073
solver:  lbfgs


Training Accuracy: 0.75052
Valid Accuracy: 0.75894
solver:  liblinear




/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


Наилучшее качество модели на валидационной выборке: 0.75894 при значениях solver=liblinear

## Шаг 4. Проверка модели на тестовой выборке <a id='4-bullet'></a>

In [106]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

In [107]:
# Дерево

tree = DecisionTreeClassifier(max_depth=7, min_samples_leaf=50, random_state=12345)
tree.fit(features_train, target_train)
tree_predicted_test = tree.predict(features_test)
print('Accuracy на test', accuracy_score(target_test, tree_predicted_test))

Accuracy на test 0.7822706065318819


In [108]:
# Лес

forest = RandomForestClassifier(max_depth=7, n_estimators=50, random_state=12345)
forest.fit(features_train, target_train)
forest_predicted_test = forest.predict(features_test)
print('Accuracy на test', accuracy_score(target_test, forest_predicted_test))

Accuracy на test 0.80248833592535


In [109]:
# Лог. регрессия

lg_reg = LogisticRegression(solver='liblinear', max_iter=4000, random_state=12345)
lg_reg.fit(features_train, target_train)
lg_reg_predicted_test = lg_reg.predict(features_test)
print('Accuracy на test', accuracy_score(target_test, lg_reg_predicted_test))

Accuracy на test 0.7402799377916018


Самый высокий результат на тестовых данных показал "Случайный лес", а самый низкий - "Логистическая регрессия"

## Шаг 5. Проверка модели на адекватность <a id='5-bullet'></a>

In [110]:
# доли классов в обучающей выборке

df_train.is_ultra.value_counts()/df_train.shape[0]*100

0    69.242739
1    30.757261
Name: is_ultra, dtype: float64

In [111]:
# доли классов в тестовой выборке

df_test.is_ultra.value_counts()/df_test.shape[0]*100

0    68.429238
1    31.570762
Name: is_ultra, dtype: float64

Доля большего класса равна 70%, что ощутимо меньше, чем accuracy любой из обученных моделей. Они успешно приспособились к дисбалансу классов в выборке.